In [1]:
import pandas as pd
import numpy as np

location = "datasets/DC_properties.csv"
df = pd.read_csv(location, dtype=None)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#avg price per sq ft
#columns - zipcode, ward, quadrant
#analysis - group by areas
#find most/least expensive areas each type, median price
#**thinking ahead--  use census info look at demographics

In [3]:
df.columns

Index(['Unnamed: 0', 'BATHRM', 'HF_BATHRM', 'HEAT', 'AC', 'NUM_UNITS', 'ROOMS',
       'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE', 'PRICE',
       'QUALIFIED', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STRUCT', 'GRADE',
       'CNDTN', 'EXTWALL', 'ROOF', 'INTWALL', 'KITCHENS', 'FIREPLACES',
       'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM', 'SOURCE', 'CMPLX_NUM',
       'LIVING_GBA', 'FULLADDRESS', 'CITY', 'STATE', 'ZIPCODE', 'NATIONALGRID',
       'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD', 'ASSESSMENT_SUBNBHD',
       'CENSUS_TRACT', 'CENSUS_BLOCK', 'WARD', 'SQUARE', 'X', 'Y', 'QUADRANT'],
      dtype='object')

In [4]:
#these columns are useful to comparing sales across location
#zip code, census tract, and census block are useful to analyze demographics of locations. Would need census data, however.
newdf = df[['PRICE', 'GBA', 'WARD', 'QUADRANT', 'ZIPCODE', 'CENSUS_TRACT', 'CENSUS_BLOCK']]
newdf.head()

,PRICE,GBA,WARD,QUADRANT,ZIPCODE,CENSUS_TRACT,CENSUS_BLOCK
0,1095000.0,2522.0,Ward 2,NW,20009.0,4201.0,004201 2006
1,NaN,2567.0,Ward 2,NW,20009.0,4201.0,004201 2006
2,2100000.0,2522.0,Ward 2,NW,20009.0,4201.0,004201 2006
3,1602000.0,2484.0,Ward 2,NW,20009.0,4201.0,004201 2006
4,NaN,5255.0,Ward 2,NW,20009.0,4201.0,004201 2006


In [5]:
newdf.rename(columns={'GBA' : 'AREA'}, inplace=True)
newdf.head()

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PRICE,AREA,WARD,QUADRANT,ZIPCODE,CENSUS_TRACT,CENSUS_BLOCK
0,1095000.0,2522.0,Ward 2,NW,20009.0,4201.0,004201 2006
1,NaN,2567.0,Ward 2,NW,20009.0,4201.0,004201 2006
2,2100000.0,2522.0,Ward 2,NW,20009.0,4201.0,004201 2006
3,1602000.0,2484.0,Ward 2,NW,20009.0,4201.0,004201 2006
4,NaN,5255.0,Ward 2,NW,20009.0,4201.0,004201 2006


In [6]:
def psf(x, y):
    return x / y

newdf['PRICE/SQFT'] = psf(newdf['PRICE'], newdf['AREA'])
newdf.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PRICE,AREA,WARD,QUADRANT,ZIPCODE,CENSUS_TRACT,CENSUS_BLOCK,PRICE/SQFT
0,1095000.0,2522.0,Ward 2,NW,20009.0,4201.0,004201 2006,434.179223
1,NaN,2567.0,Ward 2,NW,20009.0,4201.0,004201 2006,NaN
2,2100000.0,2522.0,Ward 2,NW,20009.0,4201.0,004201 2006,832.672482
3,1602000.0,2484.0,Ward 2,NW,20009.0,4201.0,004201 2006,644.927536
4,NaN,5255.0,Ward 2,NW,20009.0,4201.0,004201 2006,NaN


In [7]:
newdf = newdf.dropna()

newdf['AREA'] = newdf['AREA'].apply(lambda x: int(x))
newdf['ZIPCODE'] = newdf['ZIPCODE'].apply(lambda x: int(x))
newdf['CENSUS_TRACT'] = newdf['CENSUS_TRACT'].apply(lambda x: int(x))
newdf['PRICE/SQFT'] = newdf['PRICE/SQFT'].apply(lambda x: round(x, 2))

newdf.head()

,PRICE,AREA,WARD,QUADRANT,ZIPCODE,CENSUS_TRACT,CENSUS_BLOCK,PRICE/SQFT
0,1095000.0,2522,Ward 2,NW,20009,4201,004201 2006,434.18
2,2100000.0,2522,Ward 2,NW,20009,4201,004201 2006,832.67
3,1602000.0,2484,Ward 2,NW,20009,4201,004201 2006,644.93
5,1950000.0,5344,Ward 2,NW,20009,4201,004201 2006,364.90
7,1050000.0,2401,Ward 2,NW,20009,4201,004201 2005,437.32


In [9]:
#Ward 2 had the highest median price per sq ft. Ward 2 contains Georgetown and most of Downtown
#Ward 8 had the lowest median price per sq ft. Ward 8 is SE DC, across the river
newdf['PRICE/SQFT'].groupby(df['WARD']).median()

WARD
Ward 1    347.690
Ward 2    590.505
Ward 3    463.200
Ward 4    293.940
Ward 5    267.380
Ward 6    408.395
Ward 7    180.920
Ward 8    157.365
Name: PRICE/SQFT, dtype: float64

In [10]:
#NW quadrant has the highest median price in DC
#SE and SW are pretty similar in median price per sq ft
newdf['PRICE/SQFT'].groupby(df['QUADRANT']).median()

QUADRANT
NE    261.16
NW    375.00
SE    203.10
SW    216.20
Name: PRICE/SQFT, dtype: float64

In [11]:
#most expensive zip codes is 20007, in Georgetown, and 20037 in Dupont Circle
#least expensive zip code is in Southeast DC near the base
newdf['PRICE/SQFT'].groupby(df['ZIPCODE']).median()

ZIPCODE
20001.0    328.810
20002.0    327.115
20003.0    445.610
20005.0    422.465
20007.0    563.605
20008.0    464.170
20009.0    429.585
20010.0    335.230
20011.0    276.040
20012.0    284.260
20015.0    413.180
20016.0    465.745
20017.0    289.030
20018.0    266.260
20019.0    177.690
20020.0    171.300
20024.0    280.035
20032.0    149.520
20036.0    506.180
20037.0    571.170
20052.0    261.670
Name: PRICE/SQFT, dtype: float64